In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pickle
import regex as re
import numpy as np
import pandas as pd
from datetime import datetime

038 (septicemia),
995.91 (sepsis), 995.92 (severe sepsis), or 785.52 (septic shock).
0380
03844
03842
03811
03812
0382


In [4]:
csv_file_path = '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

In [5]:
!ls '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

ADMISSIONS.csv				  sepsis_neg_pivoted.csv
clean_sepsis_pos.csv			  SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
DIAGNOSES_ICD.csv			  SEPSIS_POS_CHART_EVENTS.csv
Sepsis_Analysis.sql			  sepsis_positive_pivoted.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
SEPSIS_NEG_CHART_EVENTS.csv		  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  SEPSIS_POS_PAT_AGE.csv
SEPSIS_NEG_PAT_AGE.csv


In [6]:
###   LOAD DATA   ###
pat  = pd.read_csv(csv_file_path+'/SEPSIS_NEG_PAT_AGE.csv')
charts  = pd.read_csv(csv_file_path+'/SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv')
labs    = pd.read_csv(csv_file_path+'/SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv')
adm = pd.read_csv(csv_file_path+'/ADMISSIONS.csv')
diag = pd.read_csv(csv_file_path+'/DIAGNOSES_ICD.csv')

In [7]:
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


get rid of these from diag 038 (septicemia), 995.91 (sepsis), 995.92 (severe sepsis), or 785.52 (septic shock). 0380 03844 03842 03811 03812 0382

In [12]:
diag.ICD9_CODE = diag.ICD9_CODE.astype(str)
# Define the conditions to filter out rows
conditions = (diag['ICD9_CODE'].str.startswith('038') |
              diag['ICD9_CODE'].str.startswith('99591') |
              diag['ICD9_CODE'].str.startswith('99592') |
              diag['ICD9_CODE'].str.startswith('78552'))

# Filter out rows that meet any of the conditions
diag_exclude = diag[conditions]

In [10]:
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651047 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


In [13]:
diag_exclude.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14182 entries, 81 to 650934
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ROW_ID      14182 non-null  int64  
 1   SUBJECT_ID  14182 non-null  int64  
 2   HADM_ID     14182 non-null  int64  
 3   SEQ_NUM     14182 non-null  float64
 4   ICD9_CODE   14182 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 664.8+ KB


In [14]:
pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45951 entries, 0 to 45950
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   subject_id  45951 non-null  int64
 1   age         45951 non-null  int64
dtypes: int64(2)
memory usage: 718.1 KB


In [15]:
pat = pat[pat.age>64]

In [16]:
pat = pat[pat.age<120]

In [17]:
pat.shape

(17861, 2)

In [18]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9640007 entries, 0 to 9640006
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   hadm_id       int64  
 2   charttime     object 
 3   lab_category  object 
 4   lab_valuenum  float64
 5   valueuom      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 441.3+ MB


In [19]:
labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274991 entries, 0 to 274990
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    274991 non-null  int64  
 1   hadm_id       274991 non-null  int64  
 2   charttime     274991 non-null  object 
 3   lab_category  274991 non-null  object 
 4   lab_valuenum  274946 non-null  float64
 5   valueuom      274973 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 12.6+ MB


In [20]:
charts_labs = pd.concat([charts,labs], ignore_index=True)

In [21]:
charts_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914998 entries, 0 to 9914997
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   hadm_id       int64  
 2   charttime     object 
 3   lab_category  object 
 4   lab_valuenum  float64
 5   valueuom      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 453.9+ MB


In [22]:
charts_labs_bu = charts_labs

In [23]:
charts_labs = charts_labs[~charts_labs['subject_id'].isin(diag_exclude['SUBJECT_ID'])]

In [24]:
charts_labs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7046234 entries, 689 to 9914997
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   hadm_id       int64  
 2   charttime     object 
 3   lab_category  object 
 4   lab_valuenum  float64
 5   valueuom      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 376.3+ MB


In [25]:
sepsis_neg = pd.merge(pat, charts_labs, on=['subject_id'], how='inner')

In [26]:
sepsis_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6527620 entries, 0 to 6527619
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   age           int64  
 2   hadm_id       int64  
 3   charttime     object 
 4   lab_category  object 
 5   lab_valuenum  float64
 6   valueuom      object 
dtypes: float64(1), int64(3), object(3)
memory usage: 348.6+ MB


In [29]:
sepsis_neg

,subject_id,age,hadm_id,charttime,lab_category,lab_valuenum,valueuom
0,6,66,107064,2175-05-30 20:30:00.000,Temperature (Fahrenheit),96.800003,Deg. F
1,6,66,107064,2175-05-30 20:30:00.000,Heart Rate,87.000000,BPM
2,6,66,107064,2175-05-30 20:30:00.000,Respiratory Rate (RR),13.000000,BPM
3,6,66,107064,2175-05-30 20:30:00.000,Non-Invasive Blood Pressure diastolic,56.000000,mmHg
4,6,66,107064,2175-05-30 21:00:00.000,Heart Rate,84.000000,BPM
...,...,...,...,...,...,...,...
6527615,99995,89,137810,2147-02-10 17:00:00.000,Respiratory Rate (RR),20.000000,insp/min
6527616,99995,89,137810,2147-02-10 17:00:00.000,Heart Rate,78.000000,bpm
6527617,99995,89,137810,2147-02-08 15:07:00.000,White Blood Cells (cells/mm^3),8.200000,K/uL
6527618,99995,89,137810,2147-02-09 02:31:00.000,White Blood Cells (cells/mm^3),7.300000,K/uL


In [30]:
sepsis_neg["lab_value"] = round(sepsis_neg["lab_valuenum"],2).astype(str)

In [31]:
sepsis_neg = sepsis_neg.drop(['lab_valuenum','valueuom'], axis=1)

In [32]:
sepsis_neg

,subject_id,age,hadm_id,charttime,lab_category,lab_value
0,6,66,107064,2175-05-30 20:30:00.000,Temperature (Fahrenheit),96.8
1,6,66,107064,2175-05-30 20:30:00.000,Heart Rate,87.0
2,6,66,107064,2175-05-30 20:30:00.000,Respiratory Rate (RR),13.0
3,6,66,107064,2175-05-30 20:30:00.000,Non-Invasive Blood Pressure diastolic,56.0
4,6,66,107064,2175-05-30 21:00:00.000,Heart Rate,84.0
...,...,...,...,...,...,...
6527615,99995,89,137810,2147-02-10 17:00:00.000,Respiratory Rate (RR),20.0
6527616,99995,89,137810,2147-02-10 17:00:00.000,Heart Rate,78.0
6527617,99995,89,137810,2147-02-08 15:07:00.000,White Blood Cells (cells/mm^3),8.2
6527618,99995,89,137810,2147-02-09 02:31:00.000,White Blood Cells (cells/mm^3),7.3


In [33]:
sepsis_neg_bkp = sepsis_neg
# sepsis_positive = sepsis_positive_bkp

In [34]:
sepsis_neg.charttime = pd.to_datetime(sepsis_neg.charttime, format = '%Y-%m-%d %H:%M:%S.%f', errors = 'coerce')

In [35]:
sepsis_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6527620 entries, 0 to 6527619
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   subject_id    int64         
 1   age           int64         
 2   hadm_id       int64         
 3   charttime     datetime64[ns]
 4   lab_category  object        
 5   lab_value     object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 298.8+ MB


In [36]:
# Find the latest test results for each patient
start = sepsis_neg[['subject_id', 'lab_category', 'charttime']].groupby(['subject_id', 'lab_category']).min().reset_index()
start.columns = ['subject_id', 'lab_category', 'charttime']
start.head(20)

,subject_id,lab_category,charttime
0,6,Heart Rate,2175-05-30 20:30:00
1,6,Invasive Blood Pressure mean,2175-05-31 07:00:00
2,6,Invasive Blood Pressure systolic,2175-05-31 07:00:00
3,6,Non-Invasive Blood Pressure diastolic,2175-05-30 20:30:00
4,6,Respiratory Rate (RR),2175-05-30 20:30:00
5,6,Temperature (Fahrenheit),2175-05-30 20:30:00
6,6,White Blood Cells (cells/mm^3),2175-05-30 16:08:00
7,12,Heart Rate,2104-08-08 04:15:00
8,12,Invasive Blood Pressure mean,2104-08-14 17:00:00
9,12,Invasive Blood Pressure systolic,2104-08-14 17:00:00


In [37]:
start['lab_category'].value_counts()

lab_category
White Blood Cells (cells/mm^3)           15211
Heart Rate                               15200
Respiratory Rate (RR)                    15188
Temperature (Fahrenheit)                  8787
Invasive Blood Pressure systolic          8549
Invasive Blood Pressure mean              8538
Non-Invasive Blood Pressure diastolic     6000
Non-Invasive Blood Pressure systolic       226
Arterial Blood Pressure diastolic          225
Arterial Blood Pressure systolic           225
Invasive Blood Pressure diastolic          204
Name: count, dtype: int64

In [38]:
# Define the list of lab categories to drop
categories_to_drop = [
    'Invasive Blood Pressure mean',
    'Invasive Blood Pressure systolic',
    'Non-Invasive Blood Pressure diastolic',
    'Invasive Blood Pressure diastolic',
    'Non-Invasive Blood Pressure systolic',
    'Arterial Blood Pressure diastolic',
    'Arterial Blood Pressure systolic',
    'WBC Count (cells/uL)'
]

# Drop rows with specified lab categories
start = start[~start['lab_category'].isin(categories_to_drop)]
start['lab_category'].value_counts()

lab_category
White Blood Cells (cells/mm^3)    15211
Heart Rate                        15200
Respiratory Rate (RR)             15188
Temperature (Fahrenheit)           8787
Name: count, dtype: int64

In [ ]:
# # Get subject_ids that have lab-category == 'Invasive Blood Pressure systolic'
# subject_ids_systolic = sepsis_positive_start[sepsis_positive_start['lab_category'] == 'Invasive Blood Pressure systolic']['subject_id'].unique()

# # Get subject_ids that have lab-category == 'Invasive Blood Pressure mean'
# subject_ids_mean = sepsis_positive_start[sepsis_positive_start['lab_category'] == 'Invasive Blood Pressure mean']['subject_id'].unique()

# # Get subject_ids that have 'Invasive Blood Pressure systolic' but not 'Invasive Blood Pressure mean'
# subject_ids_systolic_only = set(subject_ids_systolic) - set(subject_ids_mean)

# # Display the subject_ids that have 'Invasive Blood Pressure systolic' but not 'Invasive Blood Pressure mean'
# print(subject_ids_systolic_only)

In [39]:
# Group by subject_id and find the min and max charttime values
subject_id_summary = start.groupby('subject_id')['charttime'].agg(['min', 'max'])

# Calculate the time difference between min and max charttime values
subject_id_summary['time_diff'] = subject_id_summary['max'] - subject_id_summary['min']

# Check if the time difference is less than 24 hours
subject_id_summary['within_24_hours'] = subject_id_summary['time_diff'] <= pd.Timedelta('1 day')

# Merge result with original DataFrame
start = pd.merge(start, subject_id_summary[['within_24_hours']], left_on='subject_id', right_index=True, how='left')


In [40]:
start.head(30)

,subject_id,lab_category,charttime,within_24_hours
0,6,Heart Rate,2175-05-30 20:30:00,True
4,6,Respiratory Rate (RR),2175-05-30 20:30:00,True
5,6,Temperature (Fahrenheit),2175-05-30 20:30:00,True
6,6,White Blood Cells (cells/mm^3),2175-05-30 16:08:00,True
7,12,Heart Rate,2104-08-08 04:15:00,True
11,12,Respiratory Rate (RR),2104-08-08 04:15:00,True
12,12,Temperature (Fahrenheit),2104-08-08 04:15:00,True
13,12,White Blood Cells (cells/mm^3),2104-08-07 09:55:00,True
14,20,Heart Rate,2183-04-28 17:00:00,True
18,20,Respiratory Rate (RR),2183-04-28 17:00:00,True


In [41]:
start['within_24_hours'].value_counts()

within_24_hours
True     40483
False    13903
Name: count, dtype: int64

In [42]:
# Filter rows where within_24_hours is True
start = start[start['within_24_hours'] == True]

In [43]:
start['within_24_hours'].value_counts()

within_24_hours
True    40483
Name: count, dtype: int64

In [44]:
start['lab_category'].value_counts()

lab_category
White Blood Cells (cells/mm^3)    11363
Heart Rate                        11352
Respiratory Rate (RR)             11342
Temperature (Fahrenheit)           6426
Name: count, dtype: int64

In [45]:
df = sepsis_neg.merge(start, how='inner', on=['subject_id', 'lab_category', 'charttime'])

In [46]:
df.shape

(40539, 7)

In [47]:
df.head()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours
0,6,66,107064,2175-05-30 20:30:00,Temperature (Fahrenheit),96.8,True
1,6,66,107064,2175-05-30 20:30:00,Heart Rate,87.0,True
2,6,66,107064,2175-05-30 20:30:00,Respiratory Rate (RR),13.0,True
3,6,66,107064,2175-05-30 16:08:00,White Blood Cells (cells/mm^3),10.6,True
4,12,72,112213,2104-08-08 04:15:00,Respiratory Rate (RR),12.0,True


In [48]:
dup = df[df.duplicated(['subject_id', 'lab_category'])]
# df_sepsis.groupby['subject_id','age','lab_category'].duplicated().sum()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40539 entries, 0 to 40538
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   subject_id       40539 non-null  int64         
 1   age              40539 non-null  int64         
 2   hadm_id          40539 non-null  int64         
 3   charttime        40539 non-null  datetime64[ns]
 4   lab_category     40539 non-null  object        
 5   lab_value        40539 non-null  object        
 6   within_24_hours  40539 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(3), object(2)
memory usage: 1.9+ MB


In [50]:
# Identify 'hadm_id' values present in dup
hadm_ids_in_dup = set(dup['hadm_id'])

# Filter out rows from df where 'hadm_id' values are present in dup
df_filtered = df[~df['hadm_id'].isin(hadm_ids_in_dup)]

In [51]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40279 entries, 0 to 40538
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   subject_id       40279 non-null  int64         
 1   age              40279 non-null  int64         
 2   hadm_id          40279 non-null  int64         
 3   charttime        40279 non-null  datetime64[ns]
 4   lab_category     40279 non-null  object        
 5   lab_value        40279 non-null  object        
 6   within_24_hours  40279 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(3), object(2)
memory usage: 2.2+ MB


In [52]:
df_filtered[df_filtered.duplicated(['subject_id', 'lab_category'])]

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours


In [53]:
df_filtered.shape

(40279, 7)

In [54]:
# Convert column names to lowercase
adm.columns = [column.lower() for column in adm.columns]

# Display the DataFrame with lowercase column names
adm.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [55]:
# Convert the column to datetime data type
adm['admittime'] = pd.to_datetime(adm['admittime'])

In [56]:
# Merge the DataFrames on subject_id
merged_df = df_filtered.merge(adm[['hadm_id','admittime']], on ='hadm_id')

In [57]:
merged_df.head()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours,admittime
0,6,66,107064,2175-05-30 20:30:00,Temperature (Fahrenheit),96.8,True,2175-05-30 07:15:00
1,6,66,107064,2175-05-30 20:30:00,Heart Rate,87.0,True,2175-05-30 07:15:00
2,6,66,107064,2175-05-30 20:30:00,Respiratory Rate (RR),13.0,True,2175-05-30 07:15:00
3,6,66,107064,2175-05-30 16:08:00,White Blood Cells (cells/mm^3),10.6,True,2175-05-30 07:15:00
4,12,72,112213,2104-08-08 04:15:00,Respiratory Rate (RR),12.0,True,2104-08-07 10:15:00


In [58]:
# Calculate the time difference between the two date columns
time_diff = merged_df['charttime'] - merged_df['admittime']

# Check if the time difference is less than or equal to 24 hours
within_24_hours = time_diff <= pd.Timedelta('1 day')

# Add the result as a new column in the DataFrame
merged_df['within_24_hours'] = within_24_hours

# Display the DataFrame
merged_df.head()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours,admittime
0,6,66,107064,2175-05-30 20:30:00,Temperature (Fahrenheit),96.8,True,2175-05-30 07:15:00
1,6,66,107064,2175-05-30 20:30:00,Heart Rate,87.0,True,2175-05-30 07:15:00
2,6,66,107064,2175-05-30 20:30:00,Respiratory Rate (RR),13.0,True,2175-05-30 07:15:00
3,6,66,107064,2175-05-30 16:08:00,White Blood Cells (cells/mm^3),10.6,True,2175-05-30 07:15:00
4,12,72,112213,2104-08-08 04:15:00,Respiratory Rate (RR),12.0,True,2104-08-07 10:15:00


In [59]:
merged_df['within_24_hours'].value_counts()

within_24_hours
True     39294
False      985
Name: count, dtype: int64

In [60]:
# Filter rows where within_24_hours is True
df_filtered = merged_df[merged_df['within_24_hours'] == True]

In [61]:
df_filtered['within_24_hours'].value_counts()

within_24_hours
True    39294
Name: count, dtype: int64

In [62]:
df = df_filtered.pivot(index=['subject_id', 'age'], columns='lab_category', values='lab_value')
df.columns.name = None
df.reset_index(inplace=True)
df.head(50)

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,6,66,87.0,13.0,96.8,10.6
1,12,72,86.0,12.0,95.9,7.7
2,20,76,80.0,12.0,96.8,17.5
3,22,65,85.0,18.0,97.6,5.1
4,23,71,90.0,7.0,95.0,7.6
5,26,72,61.0,20.0,99.0,NaN
6,28,74,90.0,12.0,96.62,6.9
7,31,72,54.0,12.0,98.7,6.9
8,35,69,71.0,12.0,97.52,11.5
9,44,80,74.0,12.0,98.78,4.6


In [63]:
df.shape

(11280, 6)

In [64]:
# # List of columns to check for NaN values
# columns_to_check = ['Heart Rate', 'Respiratory Rate (RR)', 'Temperature (Fahrenheit)', 'White Blood Cells (cells/mm^3)']

# # Drop rows with NaN values in any of the specified columns
# df_cleaned = df.dropna(subset=columns_to_check, how='any', na_values=['NaN', 'nan'])
df.replace(['NaN', 'nan'], float('nan'), inplace=True)
df_cleaned = df.dropna()

# check shape of the cleaned DataFrame
df_cleaned.shape


(6032, 6)

In [65]:
df_cleaned

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,6,66,87.0,13.0,96.8,10.6
1,12,72,86.0,12.0,95.9,7.7
2,20,76,80.0,12.0,96.8,17.5
3,22,65,85.0,18.0,97.6,5.1
4,23,71,90.0,7.0,95.0,7.6
...,...,...,...,...,...,...
6451,32777,74,78.0,12.0,97.16,8.7
6452,32779,69,114.0,16.0,100.0,21.4
6453,32796,69,87.0,15.0,98.8,16.2
6454,32804,80,94.0,18.0,98.4,14.0


In [66]:
# Define a function to concatenate the unit strings to each value in the specified columns
def add_unit(column):
    if column.name == 'Heart Rate':
        return column.astype(str) + ' BPM'
    # elif column.name == 'Invasive Blood Pressure mean':
    #     return column.astype(str) + ' mmHg'
    elif column.name == 'Respiratory Rate (RR)':
        return column.astype(str) + ' BPM'
    elif column.name == 'Temperature (Fahrenheit)':
        return column.astype(str) + ' Deg. F'
    elif column.name == 'White Blood Cells (cells/mm^3)':
        return column.astype(str) + ' K/uL'
    else:
        return column

# Apply the function to the specified columns
df_updated = df_cleaned.apply(add_unit)

# Display the updated DataFrame
df_updated.head()

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,6,66,87.0 BPM,13.0 BPM,96.8 Deg. F,10.6 K/uL
1,12,72,86.0 BPM,12.0 BPM,95.9 Deg. F,7.7 K/uL
2,20,76,80.0 BPM,12.0 BPM,96.8 Deg. F,17.5 K/uL
3,22,65,85.0 BPM,18.0 BPM,97.6 Deg. F,5.1 K/uL
4,23,71,90.0 BPM,7.0 BPM,95.0 Deg. F,7.6 K/uL


In [67]:
# Drop the column
df_updated.drop(columns=['age','subject_id'], inplace=True)

# Display the DataFrame after dropping the column
df_updated.head()

,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,87.0 BPM,13.0 BPM,96.8 Deg. F,10.6 K/uL
1,86.0 BPM,12.0 BPM,95.9 Deg. F,7.7 K/uL
2,80.0 BPM,12.0 BPM,96.8 Deg. F,17.5 K/uL
3,85.0 BPM,18.0 BPM,97.6 Deg. F,5.1 K/uL
4,90.0 BPM,7.0 BPM,95.0 Deg. F,7.6 K/uL


In [68]:
df_updated.to_csv(csv_file_path+'/clean_sepsis_neg.csv')

In [70]:
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6032 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Heart Rate                      6032 non-null   object
 1   Respiratory Rate (RR)           6032 non-null   object
 2   Temperature (Fahrenheit)        6032 non-null   object
 3   White Blood Cells (cells/mm^3)  6032 non-null   object
dtypes: object(4)
memory usage: 235.6+ KB


In [69]:
!ls '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

ADMISSIONS.csv				  SEPSIS_NEG_PAT_AGE.csv
clean_sepsis_neg.csv			  sepsis_neg_pivoted.csv
clean_sepsis_pos.csv			  SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
DIAGNOSES_ICD.csv			  SEPSIS_POS_CHART_EVENTS.csv
Sepsis_Analysis.sql			  sepsis_positive_pivoted.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
SEPSIS_NEG_CHART_EVENTS.csv		  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  SEPSIS_POS_PAT_AGE.csv
